In [ ]:
import os

if False:
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

            import os
import re
import glob
import pathlib
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

import PIL
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from collections import Counter

from warnings import filterwarnings
filterwarnings('ignore')

SEED=42 # Also use 442, 4442, 422, 4222
np.random.seed(SEED)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Flatten, Activation, 
    MaxPooling2D, AveragePooling2D, ZeroPadding2D, GlobalAveragePooling2D, GlobalMaxPooling2D, add
)

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [ ]:
CFG = dict(
        batch_size        =  16,     # 8; 16; 32; 64; bigger batch size => memory allocation issue
        epochs            =  20,   # 5; 10; 20;
        verbose           =   1,    # 0; 1
        workers           =   4,    # 1; 2; 3

        optimizer         = 'adam', # 'SGD', 'RMSprop'

        RANDOM_STATE      =  123,   
    
        # Path to save a model
        path_model        = '../working/',

        # Images sizes - 224 for ResNet conversion (default is 32x32)
        img_size          = 224, 
        img_height        = 224, 
        img_width         = 224, 

        # Images augs
        ROTATION          = 180.0,
        ZOOM              =  10.0,
        ZOOM_RANGE        =  [0.9,1.1],
        HZOOM             =  10.0,
        WZOOM             =  10.0,
        HSHIFT            =  10.0,
        WSHIFT            =  10.0,
        SHEAR             =   5.0,
        HFLIP             = True,
        VFLIP             = True,

        # Postprocessing
        label_smooth_fac  =  0.00,  # 0.01; 0.05; 0.1; 0.2;    
)

In [ ]:
BASEPATH = "/kaggle/input/fitzpatrickdata"
df_train_full = pd.read_csv(os.path.join(BASEPATH, 'train17.csv'))
df_test  = pd.read_csv(os.path.join(BASEPATH, 'train177.csv'))
df_sub   = pd.read_csv(os.path.join(BASEPATH, 'sample_submission177.csv'))

In [ ]:
train_path = '/kaggle/input/fitzpatrick2class/Fitzpatrick2Class/Train'
test_path = '/kaggle/input/fitzpatrick2class/Fitzpatrick2Class/Test'

In [ ]:
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data directory
data_directory = '/kaggle/input/fitzpatrick2class/Fitzpatrick2Class' #NEEDS TO BE SORTED BY LABEL/TrainTest

# Define the ImageDataGenerator 
datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Rescale pixel values to the range [0, 1]
    rotation_range=20,      # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2, # Randomly shift images vertically by up to 20% of the height
    horizontal_flip=True,   # Randomly flip images horizontally
    validation_split=0.2    # Split the data into training (80%) and validation (20%)
)

# Create the training data generator
train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224), # Resize images to 224x224 pixels (adjust as needed)
    batch_size=32,          # Batch size for training
    class_mode='binary',    # Set class_mode to 'binary' or 'categorical' based on your problem
    subset='training'       # Specify that this is the training set
)

# Create the validation data generator
validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224), # Resize images to 224x224 pixels (should match training)
    batch_size=32,          # Batch size for validation
    class_mode='binary',    
    subset='validation'     
)

In [ ]:
# Count number of images in each set.
img_count_train = len(list(train_dir.glob('*/*.jpg')))
img_count_test  = len(list(test_dir.glob('*/*.jpg')))
print('{} train images'.format(img_count_train))
print('{} test  images'.format(img_count_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

data_directory = '/kaggle/input/fitzpatrick2class/Fitzpatrick2Class' #NEEDS TO BE SORTED BY LABEL

datagen = ImageDataGenerator(
    rescale=1.0/255.0,          
    rotation_range=20,          
    width_shift_range=0.2,      
    height_shift_range=0.2,     
    horizontal_flip=True,       
    zoom_range=0.2,             
    brightness_range=[0.8, 1.2] 
)

data_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),    
    batch_size=32,
    class_mode='binary',      
    shuffle=True              
)
 
base_model = VGG19(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(data_generator, epochs=20, steps_per_epoch=len(data_generator))

In [ ]:
accuracy = model.evaluate(validation_generator)[1]

In [ ]:
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
model_file_path = '/kaggle/working/.h5'
model.save(model_file_path)
print(f"Model saved to {model_file_path}")

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

model = load_model('/kaggle/working/.h5')

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Define the directory containing test images
test_image_dir = '/kaggle/input/fitzpatrick2class/Fitzpatrick2Class'

# List to store predictions and corresponding image filenames
predictions = []
image_filenames = []

# Iterate through test images
for image_filename in os.listdir(test_image_dir):
    if image_filename.endswith('.jpg'):
        image_path = os.path.join(test_image_dir, image_filename)
        # Load and preprocess the image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = cv2.resize(img, (224, 224))  # Resize to match model input size
        img = img / 255.0  # Normalize pixel values to [0, 1]
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        
        # Make a prediction
        prediction = model.predict(img)
        predictions.append(prediction[0][0])  
        image_filenames.append(image_filename)  

if len(predictions) == len(image_filenames):
    # Create a DataFrame for predictions
    df_predictions = pd.DataFrame({
        'image_name': image_filenames,
        'target': predictions
    })

    # Save the predictions to a CSV file
    df_predictions.to_csv('submission.csv', index=False)
else:
    print("Error: Predictions and image filenames have different lengths.")

In [ ]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tf-slim
!pip install opencv-python

In [ ]:
!pip install tensorflow==2.5
!pip install opencv-python-headless

!git clone https://github.com/tensorflow/models.git

import os
os.chdir('/kaggle/working/models/research')
!protoc object_detection/protos/*.proto --python_out=.

with open('object_detection/packages/tf2/setup.py', 'a') as f:
    f.write("\"tf-models-official\",\n")

!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
   --pipeline_config_path=PATH_TO_CONFIG_FILE \
   --model_dir=MODEL_DIR \
   --num_train_steps=NUM_TRAIN_STEPS \
   --sample_1_of_n_eval_examples=1 \
   --alsologtostderr
!python /kaggle/working/models/research/object_detection/exporter_main_v2.py \
   --input_type=image_tensor \
   --pipeline_config_path=PATH_TO_CONFIG_FILE \
   --trained_checkpoint_dir=MODEL_DIR \
   --output_directory=EXPORT_DIR

In [ ]:
base_model = tf.keras.applications.VGG19(
    weights='imagenet', 
    include_top=False, 
    input_tensor=input_layer
)

In [ ]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
images = images / 255.0
split_ratio = 0.8
split_index = int(len(images) * split_ratio)

train_images, test_images = images[:split_index], images[split_index:]
train_labels, test_labels = labels[:split_index], labels[split_index:]

# Define the model
input_shape = (*images.shape[1:],)
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(4, activation='linear')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')  

# Train the model
history = model.fit(train_images, train_labels, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set and calculate accuracy
loss = model.evaluate(test_images, test_labels)
accuracy = 100.0 - loss  # You can consider this as a simple accuracy metric

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy:.2f}%')